# Step 14 Using the Same Variable

If the same variable is used in the addition, it cannot get the gradient properly.

In [50]:
import numpy as np
import import_ipynb
from Codes.step13 import Variable, Function

%run Codes/step13.ipynb

In [52]:
x=Variable(np.array(3.0))
y=add(x,x)
print('y',y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


Gradient of x has to be 2, not 1. 

## 14.1 What caused the error?

x.grad=gx

The implementation that we made so far is only inserting the gradient, so when the same variable is used more than once, it is overlaying the value. 

<img src="image/step14_1.jpeg">

Gradient of x has to be 1+1=2. In other words, it has to __sum up__ the gradient values. 

## 14.2 Solution

In [53]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx #sum up the gradient value 
            
                if x.creator is not None:
                    funcs.append(x.creator)

In [54]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)

2.0


In [56]:
x=Variable(np.array(3.0))
y=add(add(x,x),x)
y.backward()
print(x.grad)

3.0


## 14.3 Resetting the Gradient

In [57]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)

y=add(add(x,x),x)
y.backward()
print(x.grad)

2.0
5.0


In [58]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] 
            gxs = f.backward(*gys) 
            if not isinstance(gxs, tuple): 
                gxs=(gxs, )
            for x, gx in zip(f.inputs, gxs): 
                if x.grad is None:
                    x.grad=gx
                else:
                    x.grad=x.grad+gx #sum up the gradient value 
            
                if x.creator is not None:
                    funcs.append(x.creator)
    def cleargrad(self):
        self.grad=None

In [59]:
x=Variable(np.array(3.0))
y=add(x,x)
y.backward()
print(x.grad)

x.cleargrad()
y=add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0


__x.cleargrad()__ : Initializing gradient